## 章節 6：決策分析（Dicision Analysis）

### 6.6 似然性函數（Likelihood）

我們來撰寫似然性函數。通常我會定義一個新的類別繼承自 Suite 類別。

In [7]:
from thinkbayes import Suite, EstimatedPdf, MakeCdfFromList, GaussianPdf


class Player(object):
    
    def __init__(self, prices, bids, diffs):
        # prices: 真實價格的序列
        # bids: 參賽者出價的序列
        # diffs: 價差的序列
        
        # 平滑化版本的真實價格分佈
        self.pdf_price = EstimatedPdf(prices)
        
        # diff 的累積機率分佈，如上圖的 CDF
        self.cdf_diff = MakeCdfFromList(diffs)
        
        mu = 0
        sigma = numpy.std(diffs)
        
        # 價差的高斯分佈
        self.pdf_error = GaussianPdf(mu, sigma)
    
    
    def ErrorDensity(self, error):
        self.pdf_error.Density(error) # 計算 error 的機率密度


class Price(Suite):
    
    def __init__(self, pmf, player):
        # pmf：先驗分佈
        # player：player 物件
        super().__init__(pmf)
        self.player = player 
        
    def Likelihood(self, data, hypo):
        price = hypo # 假設展示品的真實價格為 price
        guess = data # 參賽者在給定 price 下最好的出價價格
        
        error = price - guess
        like  = self.player.ErrorDensity(error) # 似然性計算
        return like
    

ErrorDensity 得到是一個**機率密度**，而非機率。但記得之前章節有提到似然性函數並不需要是個機率值；它只需要計算與真實機率成常數比例的數值。只要所有的似然性數值跟機率成相同的比例，最後當我們做正規化的時後這些常數會被消除掉。

所以說，機率密度函數是很適合當作一個似然性函數。